In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd

In [ ]:
from ln_utils import *

In [ ]:
%matplotlib inline

# Parameters

is_directed = True
time_window = 86400*7

# Load temporal data

In [ ]:
graph_files = []

In [ ]:
data_dir = "../LNdata/lncaptures/lngraph/2019/"
graph_files +=  [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f]
MIN_TIME = 1549065601-86400 #Saturday, February 2, 2019 12:00:01 AM
#MAX_TIME = 1552867201 #Monday, March 18, 2019 12:00:01 AM

In [ ]:
data_dir = "../LNdata/"
#graph_files = [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f]
graph_files += [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f][5:]
#MIN_TIME = 1552478399 # Wednesday, March 13, 2019 11:59:59 AM
MAX_TIME = 1553947199 # Saturday, March 30, 2019 11:59:59 AM

In [ ]:
#graph_files = graph_files[:10]
graph_files

In [ ]:
EDGE_KEYS = ["node1_pub","node2_pub","last_update","capacity","channel_id",'node1_policy','node2_policy']
nodes, edges = load_temp_data(graph_files[:-1], edge_keys=EDGE_KEYS)
print(len(nodes), len(edges))

In [ ]:
edges.dtypes

In [ ]:
edges.head(3)

In [ ]:
edges.groupby(["node1_pub","channel_id"])["node2_pub"].nunique().value_counts()

In [ ]:
nodes = nodes[(nodes["last_update"] > MIN_TIME) & (nodes["last_update"] < MAX_TIME)]
edges = edges[(edges["last_update"] > MIN_TIME) & (edges["last_update"] < MAX_TIME)]
len(nodes), len(edges)

In [ ]:
edges = edges.sort_values("last_update")

In [ ]:
def discover_changes(updates_df):
    updates_df["capacity"] = updates_df["capacity"].astype("float64")
    channel_state = {}
    channel_nodes = {}
    events = []
    for idx, row in updates_df.iterrows():
        n1p, n2p, chan_id, last_update, cap = row["node1_pub"], row["node2_pub"], row["channel_id"], row["last_update"], row["capacity"]
        is_new_channel = chan_id not in channel_state
        cap_change = 0
        if not is_new_channel:
            cap_change = cap - channel_state[chan_id]
        else:
            channel_nodes[chan_id] = (n1p,n2p)
        channel_state[chan_id] = cap
        events.append([last_update, chan_id, is_new_channel, cap, cap_change])
    return pd.DataFrame(events, columns=["time","channel_id","is_new","capacity","cap_diff"]), channel_nodes

In [ ]:
events, channel_nodes = discover_changes(edges)

In [ ]:
events["is_new"].value_counts()

In [ ]:
(events["cap_diff"] != 0).value_counts()

In [ ]:
events.head()

In [ ]:
events.to_csv("channel_events.csv", index=False)

In [ ]:
new_channels = events[events["is_new"]]
new_channels.shape

In [ ]:
new_channels["time"].hist(bins=100)

edges["last_update"].hist(bins=100)

edges.head(3)

changes = edges.groupby(["node1_pub","node2_pub","capacity","channel_id"])["last_update"].nunique().reset_index()#.value_counts()[:20]

changes.head()

loc = 1
p1 = changes.loc[loc]["node1_pub"]
p2 = changes.loc[loc]["node2_pub"]

for idx, row in edges[(edges["node1_pub"]==p1) & (edges["node2_pub"]==p2)].iterrows():
    print(row["node1_policy"])
print()
for idx, row in edges[(edges["node1_pub"]==p1) & (edges["node2_pub"]==p2)].iterrows():
    print(row["node2_policy"])